In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
food_df = pd.read_csv('../data/food_access_data.csv')
rural_county = pd.read_csv('../data/rural_county.csv', header = 1)
tn_df = pd.read_csv('../data/tn_county_data.csv', header = 1)
state_county = pd.read_csv('../data/StateAndCountyData.csv')
national_df = pd.read_csv('../data/national_data.csv', header = 1)
usa_dataset = pd.read_csv('../data/usa_dataset.csv', header = 1)

In [ ]:
rural_county.head()

In [ ]:
food_df.isna().sum()

In [ ]:
food_df.shape

In [ ]:
# only look at tennessee's food environment

tn_fa = food_df.loc[food_df['State'] == 'Tennessee']
tn_fa = tn_fa.reset_index(drop=True)
tn_fa

In [ ]:
# subset tn food access

tn_fa = tn_fa[['CensusTract','State','County','Urban','Pop2010','MedianFamilyIncome','PovertyRate','LA1and10','LATracts1','LATracts10','LATracts20',
         'LAPOP1_10','lapop1','lapop10','lapop20','TractLOWI','TractHUNV']]

In [ ]:
#cleaned up columns for tn_fa

tn_fa = tn_fa.rename(columns = {'LATracts1':'low_access_one','LATracts10':'low_access_10','LATracts20':'low_access_20',
                                'LAPOP1_10':'la_numbers','TractLOWI':'low_income_pop', 'TractHUNV':'no_vehicle'})

In [ ]:
#dropped "County" in County name == "Blount County" - "Blount"

tn_fa['County'] = tn_fa.County.str[:-7]

In [ ]:
#sum of population 10 miles away from grocery store

low_access_10 = tn_fa.groupby('County')['lapop10'].sum().to_frame().reset_index()

In [ ]:
# total population effected by a food desert

total_low_access = tn_fa.groupby('County')['la_numbers'].sum().to_frame().reset_index()

In [ ]:
#EDA of most rural counties in tn

rural_county.sort_values('% rural', ascending = False).head(20)

In [ ]:
#data for number of rural residents in tn

rural_county = rural_county[['FIPS', 'State', 'County', 'Population','# rural residents', '% rural', 'Median Household Income',
             '% Insufficient Sleep', '% Enrolled in Free or Reduced Lunch', '% Less Than 18 Years of Age']]

In [ ]:
rural_county

In [ ]:
tn_df.head()

In [ ]:
# subset of tn_df

tn_df = tn_df[['County', '% Adults with Obesity', '# Some College', '% Children in Poverty', '20th Percentile Income']]

In [ ]:
#merged low access numbers subsets

food_desert_numbers = pd.merge(low_access_10, total_low_access, left_on = 'County', right_on = 'County', how = 'inner')

In [ ]:
food_desert_numbers.head(1)

In [ ]:
#merged food deserts with rural county to start aggregations for population effected

food_deserts = pd.merge(food_desert_numbers, rural_county, left_on = 'County', right_on = 'County', how = 'inner')

In [ ]:
food_deserts.head(1)

In [ ]:
#merged all subsets to look at trends across all food deserts

trends = pd.merge(food_deserts, tn_df, left_on = 'County', right_on = 'County', how = 'inner')

In [ ]:
#renamed columns

tn_trends = trends.rename(columns = {'# rural residents': 'rural_pop', '% rural':'rural_perc', '% Insufficient Sleep': 'insufficient_sleep_perc',
                        '% Enrolled in Free or Reduced Lunch':'free_lunch_perc', '% Less Than 18 Years of Age':'under_18_perc',
                        '% Adults with Obesity':'adult_obesity_perc', '# Some College': 'some_college', '% Children in Poverty':'child_poverty_perc'})

In [ ]:
#under 18 population percentage

tn_trends['under_18_pop'] = round((tn_trends['Population'] * tn_trends['under_18_perc'])/100,0)

In [ ]:
#over 18 population percentage

tn_trends['adult_population'] = (tn_trends['Population']) - (tn_trends['under_18_pop'])

In [ ]:
#percentage of adults with some college per tennessee county

tn_trends['pop_some_college'] = round(tn_trends['some_college'] / tn_trends['adult_population'] * 100,2)

In [ ]:
tn_trends.head(1)

In [ ]:
#isolated 100% rural counties in tn

totally_rural = tn_trends.loc[tn_trends['rural_perc'] == 100.0].reset_index(drop=True)

In [ ]:
totally_rural.head(1)

In [ ]:
#defined urban counties if rural population percentage <50%

urban_trends = tn_trends.loc[tn_trends['rural_perc'] < 50.0].reset_index(drop = True)

In [ ]:
urban_trends.head(1)

In [ ]:
#There isnt a clear definition of rural according to the census. rural is determined by an area not designated as urban.
#defined rural counties if rural population percentage > 50%

rural_trends = tn_trends.loc[tn_trends['rural_perc'] > 50.0].reset_index(drop = True)

In [ ]:
national_df.head()

In [ ]:
#subset of national_df

national_df = national_df[['County','State', '% Adults with Obesity', '# Some College', '% Children in Poverty', '20th Percentile Income']]

In [ ]:
national_df.head()

In [ ]:
# subset for all the states

state_level = national_df.loc[national_df['County'].isnull()]

In [ ]:
# subset for all counties in the U.S

national_county = national_df.loc[~national_df['County'].isnull()]
national_counties = national_county[['County', 'State', '% Adults with Obesity', '# Some College', '% Children in Poverty', '20th Percentile Income']]

In [ ]:
national_counties.head()

In [ ]:
#subset of usa_dataset

all_states = usa_dataset[['FIPS', 'State', 'County', 'Population','# rural residents', '% rural', 'Median Household Income',
             '% Insufficient Sleep', '% Enrolled in Free or Reduced Lunch', '% Less Than 18 Years of Age']]

In [ ]:
#null values give just the state statistics

states = all_states.loc[all_states['County'].isnull()]

In [ ]:
# only focused on counties

all_counties = all_states.loc[~all_states['County'].isnull()]

In [ ]:
#subset of county df

all_county_subset = all_counties[['FIPS','State','County', 'Population']]

In [ ]:
#merged states and state_level, then renamed the columns the same as other df

national_stats = pd.merge(states, state_level, left_on = 'State', right_on = 'State', how = 'inner')
national_stats = national_stats.drop(columns = {'County_x', 'County_y'})
national_stats = national_stats.rename(columns = {'# rural residents': 'rural_pop', '% rural':'rural_perc', '% Insufficient Sleep': 'insufficient_sleep_perc','% Enrolled in Free or Reduced Lunch':'free_lunch_perc', '% Less Than 18 Years of Age':'under_18_perc',
                        '% Adults with Obesity':'adult_obesity_perc', '# Some College': 'some_college', '% Children in Poverty':'child_poverty_perc'})

In [ ]:
# data frame for all states to compare to Tennessee. What are the similarities/differences?

national_stats.head()

In [ ]:
tn_trends.head()

In [ ]:
state_county.head()

In [ ]:
#EDA on food access data

us_county_low_access = state_county.loc[state_county['Variable_Code'] == 'LACCESS_POP15'].reset_index(drop = True)
us_county_low_access.head(1)

In [ ]:
all_county_subset.head(1)

In [ ]:
#merged low access numbers and county numbers

us_counties = pd.merge(us_county_low_access, all_county_subset, on = 'FIPS', how = 'inner')

In [ ]:
#cleaned up df

us_counties = us_counties.drop(columns = {'State_y','County_y'}).rename(columns = {'State_x':'state','County_x':'county','Value':
                                                                    'low_acc_pop15'})

In [ ]:
#created  a column for low access percentage by population for all US counties

us_counties['low_acc_pop15'] = round(us_counties.low_acc_pop15,2)
us_counties['la_percentage'] = round(us_counties.low_acc_pop15/us_counties.Population * 100,2)
us_counties

In [ ]:
#fast food restaurants for 2016, fast-food/1000 population 2016, fast food expenditures per capita 2012

fast_food_county = state_county.loc[(state_county['Variable_Code'] == 'FFR16') | 
                                    (state_county['Variable_Code'] == 'FFRPTH16') |
                                    (state_county['Variable_Code'] == 'PC_FFRSALES12')].reset_index(drop = True)

In [ ]:
#subset of fast food per state

state_ff_count = fast_food_county.loc[fast_food_county['Variable_Code'] == 'FFR16'].drop_duplicates('State').reset_index(drop=True)
state_ff_count = state_ff_count[['FIPS','State','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_expenditures'})

In [ ]:
#percentage of fast food restaurants per 1000 pop for all states

state_ff_by_pop = fast_food_county.loc[fast_food_county['Variable_Code'] == 'FFRPTH16'].drop_duplicates('State').reset_index(drop=True)
state_ff_by_pop = state_ff_by_pop[['FIPS','State','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_expenditures'})

In [ ]:
# state fast food expenditures

state_ff_expend = fast_food_county.loc[fast_food_county['Variable_Code'] == 'PC_FFRSALES12'].drop_duplicates('State').reset_index(drop=True)
state_ff_expend = state_ff_expend[['FIPS','State','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_expenditures'})

In [ ]:
#merged the three subsets of state df's together to have only 95 rows

state_food = pd.merge(state_ff_count, state_ff_by_pop, on = 'State', how = 'inner')

In [ ]:

state_ff = pd.merge(state_food, state_ff_expend, on = 'State', how = 'inner')
state_ff = state_ff.drop(columns = {'FIPS_y', 'FIPS'}).rename(columns = {'FIPS_x':'FIPS','ff_expenditures_x':'ff_16', 'Value_x':'ff_count',
                                                             'ff_expenditures_y':'ff_1k', 'Value_y':'ff_percent','Value':'ff_dollars'})

In [ ]:
state_ff.head()

In [ ]:
#pulled tn out of fast_food df

tn_ff = fast_food_county.loc[fast_food_county['State'] == 'TN'].reset_index(drop=True)

In [ ]:
tn_ff.head()

In [ ]:
# tried a loop on the variable code, but had too many NaN values


# for index, row in tn_ff.iterrows():
#     if row.Variable_Code == "FFR16":
#         tn_ff.loc[index, 'fast_food16'] = tn_ff.loc[index, 'Value']
#     elif row.Variable_Code == "FFRPTH16":
#         tn_ff.loc[index, 'ff_1k'] = tn_ff.loc[index, 'Value']
#     elif row.Variable_Code == "PC_FFRSALES12":
#         tn_ff.loc[index, "ff_expenditures"] = tn_ff.loc[index, 'Value']
#     else:
#         tn_ff.loc[index, "fast_food16"] = -1

In [ ]:
#subset data for each individual variable code

tn_ff_count = tn_ff.loc[tn_ff['Variable_Code'] == 'FFR16'].reset_index(drop=True)
tn_ff_count = tn_ff_count[['FIPS','State','County','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_count'})
#tn_ff_count

In [ ]:
#tennessee fast food percentage per 1000 pop

tn_ff_perc = tn_ff.loc[tn_ff['Variable_Code'] == 'FFRPTH16'].reset_index(drop=True)
tn_ff_perc = tn_ff_perc[['FIPS','State','County','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_1k', 'Value': 'ff_percent'})
#tn_ff_perc

In [ ]:
#expenditures is by state, cant look at it by county for Tennessee

# tn_ff_expend = tn_ff.loc[tn_ff['Variable_Code'] == 'PC_FFRSALES12'].reset_index(drop=True)
# tn_ff_expend = tn_ff_expend[['FIPS','State','County','Variable_Code','Value']].rename(columns = {'Variable_Code':'ff_expenditures', 'Value': 'ff_dollars'})
# #tn_ff_expend

In [ ]:
#merged the three subsets together

tn_county_ff = pd.merge(tn_ff_count, tn_ff_perc, on = 'County', how = 'inner')

In [ ]:
# tn_county_ff = pd.merge(tn_county_ff, tn_ff_expend, on = 'County', how = 'inner')

In [ ]:
#dropped druplicate columns after merge

tn_county_ff = tn_county_ff.drop(columns = {'FIPS_y','State_y'})

In [ ]:
tn_county_ff.head(1)

In [ ]:
# export to csv for tn_counties notebook
# tn_county_ff.to_csv('../data/tn_ff.csv', index = False)

In [ ]:
#df of all Tennessee food environment statistics

tn_stats = state_county.loc[state_county['State'] == 'TN']

In [ ]:
#df of grocery, supercenters, and specialized markets

tn_grocery = tn_stats.loc[(tn_stats['Variable_Code'] == 'GROC16') | (tn_stats['Variable_Code'] == 'SUPERC16')|
                         (tn_stats['Variable_Code'] == 'SPECS16')].reset_index(drop=True)
#tn_grocery.to_csv('../data/tn_grocery.csv', index = False)

In [ ]:
tn_grocery.head(1)

In [ ]:
#sum of all stores by Tennessee County

tn_groc_count = tn_grocery.groupby(['County'])['Value'].sum().to_frame().reset_index()
tn_groc_count

In [ ]:
#merged fast food restaurants and grocery store count for all Tennessee Counties

tn_county_resources = pd.merge(tn_county_ff, tn_groc_count, on = 'County', how = 'inner').rename(columns = {'FIPS_x':'FIPS', 'State_x':'state','Value_x':'ff_count','Value_y':'groc_store_count'})

In [ ]:
tn_county_resources.head(1)

In [ ]:
#same as grocery stores. Pulled farm variables from dataframe

tn_farms = tn_stats.loc[(tn_stats['Variable_Code'] == 'FMRKT13') | (tn_stats['Variable_Code'] == 'FRESHVEG_FARMS12') | 
              (tn_stats['Variable_Code'] == 'BERRY_FARMS12') | (tn_stats['Variable_Code'] == 'ORCHARD_FARMS12')].reset_index(drop = True)

In [ ]:
#total farm count by Tennessee county

tn_farm_count = tn_farms.groupby(['County'])['Value'].sum().to_frame().reset_index()
tn_farm_count.head(1)

In [ ]:
#merged farm counts to rest of Tennesse county resource table

tn_county_resources = pd.merge(tn_county_resources, tn_farm_count, on = 'County', how = 'inner').rename(columns = {'Value':'farm_count'})

In [ ]:
#exported for tn_counties notebook

# tn_county_resources.to_csv('../data/tn_farms.csv', index = False)

In [ ]:
#EDA of Trousdale County. Missing some data that was impacting an attempted merge

state_county.loc[state_county['County'] == 'Trousdale']

In [ ]:
#df of tennessee county population with no vehicle and low access to food

la_no_vehicle = state_county.loc[(state_county['State'] == 'TN') & (state_county['Variable_Code'] == 'LACCESS_HHNV15')]

In [ ]:
#df of low access population for 2015

la_2015= state_county.loc[(state_county['State'] == 'TN') & (state_county['Variable_Code'] == 'LACCESS_POP15')]

In [ ]:
#merged no vehicle to tn_trends df

tn_trends = pd.merge(tn_trends, la_no_vehicle, on= 'County', how ='inner')

In [ ]:
#merged low access pop 2015 to rest of tn_trends df
tn_trends = pd.merge(tn_trends, la_2015, on = 'County', how = 'left')

In [ ]:
#cleaned up tn_trends df
tn_trends = tn_trends.drop(columns = {'FIPS_y', 'State_y', 'FIPS_x', 'State_x','Variable_Code_x', 'Variable_Code_y' }).rename(columns = {'lapop10':'lapop_10mile','la_numbers':'la_2010',
                                                                                     'Value_x':'la_no_car_count','Value_y':'la_2015_count'})

In [ ]:
#percentage of low access population for 2010

tn_trends['low_access_percent'] = tn_trends.la_2010/tn_trends.Population * 100

In [ ]:
food_df.head(1)

In [ ]:
#sum of population not within 10 miles of a grocery store per state

state_lapop_10miles = food_df.groupby('State')['lapop10'].sum().to_frame().reset_index()
state_lapop_10miles

In [ ]:
#merged population not within 10 miles of a grocery store with national df

national_stats = pd.merge(national_stats, state_lapop_10miles, on = 'State', how = 'inner')

In [ ]:
national_stats.head(1)

In [ ]:
rural_trends.head(1)

In [ ]:
urban_trends.head(1)

In [ ]:
tn_county_ff.head(1)

In [ ]:
tn_trends.head(1)

In [ ]:
tn_county_resources.head(1)

In [ ]:
national_stats.head(1)

In [ ]:
state_ff.head(1)

In [ ]:
#exporting for dashboards
rural_trends.to_csv('../data/dashboard_csv/rural_food_trends.csv', index = False)
urban_trends.to_csv('../data/dashboard_csv/urban_food_trends.csv', index = False)
tn_trends.to_csv('../data/dashboard_csv/all_tn_county_trends.csv', index = False)
#tn_county_ff.to_csv('../data/dashboard_csv/tn_ff.csv', index = False)
tn_county_resources.to_csv('../data/dashboard_csv/tn_resources.csv', index = False)
national_stats.to_csv('../data/dashboard_csv/national_stats.csv', index = False)
state_ff.to_csv('../data/dashboard_csv/state_ff.csv', index = False)
us_counties.to_csv('../data/dashboard_csv/us_counties.csv', index = False)